### Maximal Matching

In [2]:
import sys
from os import path
sys.path.insert(0, '../src')
import one_to_n

import datetime
import textdistance
import editdistance
import pandas as pd
import networkx as nx
import re

In [3]:
table_a = one_to_n.lat_convert_df("../Amazon-GoogleProducts/Amazon.csv")

table_b = one_to_n.lat_convert_df("../Amazon-GoogleProducts/GoogleProducts.csv")

now = datetime.datetime.now()
bipartite_graph_result = one_to_n.valcomp_treshold_updated_maximal_construct_graph(table_a, table_b, "title", 0.5)
timing_tresh = (datetime.datetime.now()-now).total_seconds()
print("---- Timing for Graph Construction with Treshold Constraint ----")
print(timing_tresh,"seconds")

2.27% complete
4.55% complete
6.82% complete
9.1% complete
11.37% complete
13.65% complete
15.92% complete
18.19% complete
20.47% complete
22.74% complete
25.02% complete
27.29% complete
29.57% complete
31.84% complete
34.11% complete
36.39% complete
38.66% complete
40.94% complete
43.21% complete
45.49% complete
47.76% complete
50.03% complete
52.31% complete
54.58% complete
56.86% complete
59.13% complete
61.4% complete
63.68% complete
65.95% complete
68.23% complete
70.5% complete
72.78% complete
75.05% complete
77.32% complete
79.6% complete
81.87% complete
84.15% complete
86.42% complete
88.7% complete
90.97% complete
93.24% complete
95.52% complete
97.79% complete
---- Timing for Graph Construction with Treshold Constraint ----
17.807568 seconds


In [4]:
def SUM_edit_edge_weight(bip_graph):
    for u,v,d in bip_graph.edges(data=True):
        val_tuple_1 = u.split("_")
        val_tuple_2 = v.split("_")
        
        val1 = re.sub("[^0-9]", "", val_tuple_1[2])
        val2 =re.sub("[^0-9]", "", val_tuple_2[2])

        d['weight'] = float(val1) + float(val2)

    return bip_graph

sum_weighted_graph = SUM_edit_edge_weight(bipartite_graph_result)

In [6]:
print("\n\n 'SUM' MAXIMAL MATCHING:")
now = datetime.datetime.now()
matching_set_maximal = nx.algorithms.matching.max_weight_matching(sum_weighted_graph)
timing_match = (datetime.datetime.now()-now).total_seconds()
print("---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----")
print(timing_match,"seconds")
print("The number of edges in the graph is:", sum_weighted_graph.number_of_edges(), "\n")


# print("The Maximal Matching Set is:", matching_set_maximal, "\n")



 'SUM' MAXIMAL MATCHING:
---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----
3.837205 seconds
The number of edges in the graph is: 1215 



### Minimal Matching

In [5]:
data_edge = sum_weighted_graph.edges()
for i in data_edge:
    first = i[0].split("_")[1]
    second = i[1].split("_")[1]
    print((first, second))

('clickart 950 000 - premier image pack (dvd-rom)', 'clickart 950000 - premier image pack (dvd-rom)')
('peachtree by sage premium accounting for nonprofits 2007', 'sage (ptree) - vernfp2007rt - premium accounting for nonprofits 2007')
("mia's math adventure: just in time", "kutoka interactive 61208 mia's math adventure: just in time!")
('spy sweeper spanish', 'webroot spy sweeper spanish edition (pc)')
('omnioutliner professional 3.0', 'omnioutliner professional 3.0 for mac')
('world book encyclopedia 2006', 'world book 2006')
('musicalis guitar workshop', 'musicalis universal guitar workshop')
('the printshop 20 professional publisher', 'printshop 20 pro publisher')
('the printshop 20 professional publisher', 'the printshop 20')
('the printshop 20', 'the printshop 20')
('data protection suite', 'data protection security suite 2005')
('kids power fun for girls', 'kids power fun for girls')
('kids power fun for girls', 'kids power fun for little ones')
('kids power fun for girls', 'kids

# ------------------------------------------------------------------------------

# Minimal Matching (Algorithm Using Max Matching)

In [9]:
def minimal_matching(sum_weighted_graph):

    new_graph = sum_weighted_graph.copy()
    max_weight = max([d['weight'] for u,v,d in new_graph.edges(data=True)])
    for u,v,d in new_graph.edges(data=True):
        d['weight'] = max_weight - d['weight']

    matching_set_minimal = nx.algorithms.matching.max_weight_matching(new_graph)
    return matching_set_minimal

In [11]:
print("\n\n 'SUM' MINIMAL MATCHING RESULTS:")
print(nx.bipartite.is_bipartite(sum_weighted_graph))
now = datetime.datetime.now()
matching_set_minimal = minimal_matching(sum_weighted_graph)
timing_match = (datetime.datetime.now()-now).total_seconds()
print("The Minimal Matching Set is:", matching_set_minimal, "\n")
print("---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----")
print(timing_match,"seconds")
print("The number of edges in the graph is:", sum_weighted_graph.number_of_edges(), "\n")



 'SUM' MINIMAL MATCHING RESULTS:
True
The Minimal Matching Set is: {('http://www.google.com/base/feeds/snippets/16709319501877103249_csdc iremember digital scrapbooking 2.0_35.99_2', 'b000hlt57q_iremember digital scrapbooking 2.0_39.95_1'), ('b000nkj29c_finale allegro 2007_199.95_1', 'http://www.google.com/base/feeds/snippets/18118065911754783035_emedia finale allegro_169.99_2'), ('http://www.google.com/base/feeds/snippets/14783944613059032198_steinberg cubase 4 software music production software_799.99_2', 'b000k6crb4_steinberg cubase 4 digital audio software_999.99_1'), ('http://www.google.com/base/feeds/snippets/18417354130985846422_autocad 2007 coursewarecustomizing the interface_39.69_2', 'b000hkgj8k_autocad 2007 courseware customizing the interface_47.95_1'), ('b0006b63rw_the sims 2 (mac)_49.99_1', 'http://www.google.com/base/feeds/snippets/12911813140487249020_aspyr the sims 2_45.99_2'), ('http://www.google.com/base/feeds/snippets/13389337162643628213_encore software 10636 - 3

# ------------------------------------------------------------------------------

In [ ]:
nx.is_connected(sum_weighted_graph)

In [85]:
"""

Constructs a maximal bipartite graph of the given two tables according to the treshold similarity.
The bipartite matching graph only includes those that have passed a certain similarity treshold.
The similarity metric takes into account the **values** in this implementation

Input: Any 2 files in any format
Output: A Bipartite Graph with Maximal Weights
"""
def edited_valcomp_construct_graph(file_one, file_n, col_to_dup, treshold_decimal):
    table_a_unprocessed = one_to_n.convert_df(file_one)
    table_b_unprocessed = one_to_n.convert_df(file_n)
    bipartite_graph = nx.Graph()
    
    table_a_unprocessed = one_to_n.create_duplicates(table_a_unprocessed, col_to_dup, 3) # Assuming that the user inputs 3 duplicates

    table_a = one_to_n.make_dict(table_a_unprocessed)
    table_b = one_to_n.make_dict(table_b_unprocessed)

    i=0
    
    for key1, val1 in table_a.items():
        comp_point_1 = val1[0].split("_")[0]

        id1 = str(key1) + '_'+ str(comp_point_1) + '_1'
        for key2, val2 in table_b.items():

            comp_point_2 = val2[0]
            dist = one_to_n.calc_jaccard(str(comp_point_1).lower(),str(comp_point_2).lower())
            i+=1
            # print("first is: ", comp_point_1, "second is:", comp_point_2, "distance is:", dist)
            if i%100000 == 0:
                print(str(round(100*i/len(file_one)/len(file_n),2))+'% complete')
#             if dist >= treshold_decimal:
                
                #add value to identifier to disitnguish two entries with different values
            id2 = str(key2) + '_' + str(comp_point_2) + '_2'
                
            num1 = re.sub("[^0-9]", "", str(val1[3]))
            num2 =re.sub("[^0-9]", "", str(val2[3]))
                
            add_weight = float(num1) + float(num2)
                
            bipartite_graph.add_edge(id1, id2, weight=add_weight)
                #edit distance and weight should be inv. prop.
                #also adding 1 to denom. to prevent divide by 0
                # add 1,2 to distinguish two key-value tuples belonging to different tables
#             else:
#                 continue
            
    return bipartite_graph


In [86]:
now = datetime.datetime.now()
bipartite_graph_result = edited_valcomp_construct_graph(table_a, table_b, "title", 0.5)
timing_tresh = (datetime.datetime.now()-now).total_seconds()
print("---- Timing for Graph Construction with Treshold Constraint ----")
print(timing_tresh,"seconds")

2.27% complete
4.55% complete
6.82% complete
9.1% complete
11.37% complete
13.65% complete
15.92% complete
18.19% complete
20.47% complete
22.74% complete
25.02% complete
27.29% complete
29.57% complete
31.84% complete
34.11% complete
36.39% complete
38.66% complete
40.94% complete
43.21% complete
45.49% complete
47.76% complete
50.03% complete
52.31% complete
54.58% complete
56.86% complete
59.13% complete
61.4% complete
63.68% complete
65.95% complete
68.23% complete
70.5% complete
72.78% complete
75.05% complete
77.32% complete
79.6% complete
81.87% complete
84.15% complete
86.42% complete
88.7% complete
90.97% complete
93.24% complete
95.52% complete
97.79% complete
---- Timing for Graph Construction with Treshold Constraint ----
46.867456 seconds


In [88]:
# print(bipartite_graph_result.edges())
# edited_weight_graph = SUM_edit_edge_weight(bipartite_graph_result)
# print(nx.number_connected_components(bipartite_graph_result))
# print(list(nx.connected_components(bipartite_graph_result)))
# print(list(bipartite_graph_result.edges()))
# print(list(nx.connected_components(bipartite_graph_result)))
# remaining = bipartite_graph_result.edges() - nx.connected_components(bipartite_graph_result)
# print("The number of edges in the graph is:", bipartite_graph_result.number_of_edges(), "\n")
# print("\n\n 'SUM' MINIMAL MATCHING RESULTS:")
# print(nx.bipartite.is_bipartite(bipartite_graph_result))
print(nx.is_connected(bipartite_graph_result))
now = datetime.datetime.now()
matching_set_minimal = nx.algorithms.bipartite.matching.minimum_weight_full_matching(bipartite_graph_result)
timing_match = (datetime.datetime.now()-now).total_seconds()
print("The Minimal Matching Set is:", matching_set_minimal, "\n")
print("---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----")
print(timing_match,"seconds")


True
The Minimal Matching Set is: {'b000egwc42_microsoft office & windows training_1': 'http://www.google.com/base/feeds/snippets/549551716004314019_ilumina expansion series: left behind on cd-rom_2', 'b0002qnd2y_form workshop 1200_1': 'http://www.google.com/base/feeds/snippets/18425416409544307946_encore 10025 hoyle card games for pc_2', 'b000hlt55i_pcdefense_1': 'http://www.google.com/base/feeds/snippets/18401771876647547599_emc auth mgr mnt ent/std9x5 2m per/u 20005-25000u - aut0025000es2_2', 'b000cbqbby_boomerang webshop platinum_1': 'http://www.google.com/base/feeds/snippets/18394561390528794760_panda software upg to gatdef performa 8200 2k0 1yr - uabpgdpcavo-gov_2', 'b000g80lqo_peachtree by sage premium accounting for nonprofits 2007_1': 'http://www.google.com/base/feeds/snippets/18437163602958068440_panda software upg to gatdef performa 8200 4k0 1yr - uabpgdpcavs-edu_2', 'b0009suw4w_hoyle card games (jewel case)_1': 'http://www.google.com/base/feeds/snippets/18422752499559886630

In [63]:
print(len(remaining))
print(inside)

0
[]


In [84]:
# print(list(bipartite_graph_result.edges()))

[('b000jz4hqo_clickart 950 000 - premier image pack (dvd-rom)_1', 'http://www.google.com/base/feeds/snippets/18441480711193821750_clickart 950000 - premier image pack (dvd-rom)_2'), ('b000g80lqo_peachtree by sage premium accounting for nonprofits 2007_1', 'http://www.google.com/base/feeds/snippets/18441188461196475272_sage (ptree) - vernfp2007rt - premium accounting for nonprofits 2007_2'), ("b0000dbykm_mia's math adventure: just in time_1", "http://www.google.com/base/feeds/snippets/18434163358494173665_kutoka interactive 61208 mia's math adventure: just in time!_2"), ('b000i82j80_spy sweeper spanish_1', 'http://www.google.com/base/feeds/snippets/12086037741590312489_webroot spy sweeper spanish edition (pc)_2'), ('b000in6u62_omnioutliner professional 3.0_1', 'http://www.google.com/base/feeds/snippets/18434616791723899127_omnioutliner professional 3.0 for mac_2'), ('b000bcz8ng_world book encyclopedia 2006_1', 'http://www.google.com/base/feeds/snippets/3077454195131808819_world book 200